In [218]:
# configuración para notebook con instalación LOCAL
FUENTES_DIR  = '../Fuentes'     # carpeta donde se encuentran archivos .py auxiliares
DATOS_DIR    = '../Datos/'      # carpeta donde se encuentran los datasets
MODELOS_DIR  = '../Modelos/'    # carpeta donde se encuentran los modelos
ARCHIVOs_DIR = '../Archivos/'   # carpeta recuperar o copiar archivos

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

In [219]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from matplotlib import pylab as plt

from grafica import *
from ClassNeuronaLineal import NeuronaLineal
import seaborn as sns

In [220]:
import pandas as pd      
import chardet


nombre_archivo = DATOS_DIR + 'automobile-simple.csv' 

#-- detectando la codificación de caracteres usada ----
with open(nombre_archivo, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large

# recupera el archivo en un objeto dataframe de pandas utilizando la codificación detectada
df = pd.read_csv(nombre_archivo, encoding=result['encoding'])

# visualiza el dataframe

df

,make,fuel-type,num-of-doors,body-style,curb-weight,engine-size,horsepower,city-mpg,highway-mpg,price,volume,eco-rating
0,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,13495.0,528019.904,33.297462
1,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,16500.0,528019.904,33.297462
2,alfa-romero,gas,two,hatchback,2823,152,154.0,19,26,16500.0,587592.640,30.898272
3,audi,gas,four,sedan,2337,109,102.0,24,30,13950.0,634816.956,42.697819
4,audi,gas,four,sedan,2824,136,115.0,18,22,17450.0,636734.832,27.997459
...,...,...,...,...,...,...,...,...,...,...,...,...
200,volvo,gas,four,sedan,2952,141,114.0,23,28,16845.0,721961.760,38.354298
201,volvo,gas,four,sedan,3049,141,160.0,19,25,19045.0,720913.920,32.043335
202,volvo,gas,four,sedan,3012,173,134.0,18,23,21485.0,721961.760,29.806494
203,volvo,diesel,four,sedan,3217,145,106.0,26,27,22470.0,721961.760,35.073342


In [221]:
df= df.dropna()
df.isnull().sum()

make            0
fuel-type       0
num-of-doors    0
body-style      0
curb-weight     0
engine-size     0
horsepower      0
city-mpg        0
highway-mpg     0
price           0
volume          0
eco-rating      0
dtype: int64

In [222]:
media_ecorating = df['eco-rating'].mean()
df['es-ecologico'] = df['eco-rating'].apply(lambda x: 1 if x > media_ecorating else 0)
df.head()

C:\Users\ivanb\AppData\Local\Temp\ipykernel_10824\620132413.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['es-ecologico'] = df['eco-rating'].apply(lambda x: 1 if x > media_ecorating else 0)


,make,fuel-type,num-of-doors,body-style,curb-weight,engine-size,horsepower,city-mpg,highway-mpg,price,volume,eco-rating,es-ecologico
0,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,13495.0,528019.904,33.297462,0
1,alfa-romero,gas,two,convertible,2548,130,111.0,21,27,16500.0,528019.904,33.297462,0
2,alfa-romero,gas,two,hatchback,2823,152,154.0,19,26,16500.0,587592.640,30.898272,0
3,audi,gas,four,sedan,2337,109,102.0,24,30,13950.0,634816.956,42.697819,1
4,audi,gas,four,sedan,2824,136,115.0,18,22,17450.0,636734.832,27.997459,0


In [223]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 197 entries, 0 to 204
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   make          197 non-null    object 
 1   fuel-type     197 non-null    object 
 2   num-of-doors  197 non-null    object 
 3   body-style    197 non-null    object 
 4   curb-weight   197 non-null    int64  
 5   engine-size   197 non-null    int64  
 6   horsepower    197 non-null    float64
 7   city-mpg      197 non-null    int64  
 8   highway-mpg   197 non-null    int64  
 9   price         197 non-null    float64
 10  volume        197 non-null    float64
 11  eco-rating    197 non-null    float64
 12  es-ecologico  197 non-null    int64  
dtypes: float64(4), int64(5), object(4)
memory usage: 21.5+ KB


In [224]:
#Hacemos todas las columnas nnumericas
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df[['make', 'fuel-type', 'num-of-doors','body-style']] = df[['make', 'fuel-type', 'num-of-doors','body-style']].apply(lambda col: label_encoder.fit_transform(col))

df.head()

C:\Users\ivanb\AppData\Local\Temp\ipykernel_10824\635271464.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['make', 'fuel-type', 'num-of-doors','body-style']] = df[['make', 'fuel-type', 'num-of-doors','body-style']].apply(lambda col: label_encoder.fit_transform(col))


,make,fuel-type,num-of-doors,body-style,curb-weight,engine-size,horsepower,city-mpg,highway-mpg,price,volume,eco-rating,es-ecologico
0,0,1,1,0,2548,130,111.0,21,27,13495.0,528019.904,33.297462,0
1,0,1,1,0,2548,130,111.0,21,27,16500.0,528019.904,33.297462,0
2,0,1,1,2,2823,152,154.0,19,26,16500.0,587592.640,30.898272,0
3,1,1,0,3,2337,109,102.0,24,30,13950.0,634816.956,42.697819,1
4,1,1,0,3,2824,136,115.0,18,22,17450.0,636734.832,27.997459,0


# Con Sigmoid

In [225]:
datos  = np.array(df)
Y = datos[:,12].reshape(-1,1) #variable que quiero predecir
X = np.delete(datos, [12], 1) #elimino la variable target

#Normalizamos
normalizador = preprocessing.StandardScaler()
X = normalizador.fit_transform(X)

#Entrenamos
from ClassNeuronaGral import NeuronaGradiente
modelo_sigmoid = NeuronaGradiente(alpha=0.01, n_iter=100, FUN='sigmoid')
modelo_sigmoid.fit(X,Y)

In [226]:
pred  = modelo_sigmoid.predict(X)
acc = modelo_sigmoid.accuracy(X, Y)
print(acc)

98.69543147208122


# Con tangente Hip

In [227]:
modelo_tanh = NeuronaGradiente(alpha=0.01, n_iter=100, FUN='tanh')
modelo_tanh.fit(X,Y)

In [228]:
pred  = modelo_tanh.predict(X)
acc = modelo_tanh.accuracy(X, Y)
print(acc)

68.9238578680203
